# Build a content-based file tagging system using Amazon Bedrock Data Automation

[Amazon S3](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html) is a popular object storage service on AWS. You can store any type of file as an object in a S3 bucket. Although you can write files of a specific type or context within a specific directory structure (path) in S3, it will be useful to add metadata to the files like it's content description, owner, context etc. so you can easily retrieve the file that you are looking for. There are two ways to do this:

**Option 1: Use the user-defined metadata feature in S3**

While uploading an object to a S3 bucket, you can optionally assign user-defined metadata as key-values pairs to the object. This will be stored along with the object. This cannot be added later on to an existing object. The only way to modify object metadata is to make a copy of the object and set the metadata.

**Option 2: Store the metadata in an external system with a reference to the object in S3**

If you want to set metadata to an existing object in S3 without copying that object or if you want to add to an existing metadata system that already exist, then it will make sense to store the metadata in an external system, like an Amazon DynamoDB table for example. This option is also applicable if the data is stored outside S3 and needs to be tagged with metadata.

In both of these options, if you do not know the metadata that describes the data stored in the object, then, you have to read the object, analyze it's content and generate the appropriate metadata. This is where AI can help.

[Amazon Bedrock Data Automation](https://aws.amazon.com/bedrock/bda/) (BDA) is a fully managed service that simplifies the process of extracting valuable insights from unstructured content—such as documents, images, video, and audio. BDA leverages generative AI to automate the transformation of multi-modal data into structured formats, enabling developers to build applications and automate complex workflows with greater speed and accuracy.

This notebook will walk you through the process of analyzing document and image files stored in S3 using [Amazon Bedrock Data Automation](https://aws.amazon.com/bedrock/bda/), generating metadata based on the content of the files and storing them as key-value pairs (tags) in an [Amazon DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Introduction.html) table with a reference to the files in S3. In the process, you will learn how to setup and interact with Amazon Bedrock Data Automation environment and configure security permissions.

<div class="alert alert-block alert-info">
<b>Note:</b>
    <ul>
        <li>This notebook should only be run from within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html">Amazon SageMaker Notebook instance</a> or within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated.html">Amazon SageMaker Studio Notebook</a>.</li>
        <li>At the time of writing this notebook, Amazon Bedrock Data Automation (BDA) was available as a preview release in the us-west-2 (Oregon) Region</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to this Region. Follow the guidance in the <i>Organize imports</i> section of this notebook.</li>
        <li>At the time of writing this notebook, BDA supported only <b>PDF, JPEG, PNG, TIFF</b> file extensions for Document/Image Modality. So this notebook will ignore the processing of any other file present in the input S3 bucket.</li>
        <li>This notebook expects files to be readily available in the specified input S3 bucket.</li>
        <li>This notebook is recommended to be run with a minimum instance size of <i>ml.t3.medium</i> and
            <ul>
                <li>With <i>Amazon Linux 2, Jupyter Lab 4</i> as the platform identifier on an Amazon SageMaker Notebook instance.</li>
                <li> (or)
                <li>With <i>Data Science 3.0</i> as the image on an Amazon SageMaker Studio Notebook.</li>
            <ul>
        </li>
        <li>At the time of this writing, the most relevant latest version of the Kernel for running this notebook,
            <ul>
                <li>On an Amazon SageMaker Notebook instance was <i>conda_python3</i></li>
                <li>On an Amazon SageMaker Studio Notebook was <i>Python 3</i></li>
            </ul>
        </li>
    </ul>
</div>

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)

    1. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    2. [Install required software libraries](#Install%20required%20software%20libraries)
    
    3. [Configure logging](#Configure%20logging)
        
        1. [System logs (Optional)](#Configure%20system%20logs%20(Optional))
        
        2. [Application logs](#Configure%20application%20logs)
    
    4. [Organize imports](#Organize%20imports)
    
    5. [Set AWS Region and boto3 config](#Set%20AWS%20Region%20and%20boto3%20config)
    
    6. [Create common objects](#Create%20common%20objects)
    
    7. [Check and create the metadata table](#Check%20and%20create%20the%20metadata%20table)
    
    8. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)
  
    9. [Setup BDA](#Setup%20BDA)

 2. [Generate and store the file tags in the metadata table](#Generate%20and%20store%20the%20file%20tags%20in%20the%20metadata%20table)
 
 3. [Retrieve the file tags from the metadata table](#Retrieve%20the%20file%20tags%20from%20the%20metadata%20table)
 
 4. [Cleanup](#Cleanup)
 
 5. [Conclusion](#Conclusion)
 
 6. [Frequently Asked Questions (FAQs)](#FAQs)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

Check and complete the prerequisites.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>
This notebook requires outbound access to the Internet to download the required software updates and to download the dataset.  You can either provide direct Internet access (default) or provide Internet access through an [Amazon VPC](https://aws.amazon.com/vpc/).  For more information on this, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html).

### B. Install required software libraries <a id ='Install%20required%20software%20libraries'> </a>
This notebook requires the following libraries:
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [Python 3.10.x](https://www.python.org/downloads/release/python-3100/)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)

Run the following cell to install the required libraries.

<div class="alert alert-block alert-warning">  
    <b>Note:</b> At the end of the installation, the Kernel will be forcefully restarted immediately. Please wait 10 seconds for the kernel to come back before running the next cell.
</div>

In [ ]:
!pip install boto3==1.35.94
!pip install sagemaker==2.237.1

import IPython

IPython.Application.instance().kernel.do_shutdown(True)

### C. Configure logging <a id ='Configure%20logging'> </a>

####  a. System logs (Optional) <a id='Configure%20system%20logs%20(Optional)'></a>

System logs refers to the logs generated by the notebook's interactions with the underlying notebook instance. Some examples of these are the logs generated when loading or saving the notebook.

These logs are automatically setup when the notebook instance is launched.

These logs can be accessed through the [Amazon CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html) console in the same AWS Region where this notebook is running.
* When running this notebook in an Amazon SageMaker Notebook instance, navigate to the following location,
    * <i>CloudWatch > Log groups > /aws/sagemaker/NotebookInstances > {notebook-instance-name}/jupyter.log</i>
* When running this notebook in an Amazon SageMaker Studio Notebook, navigate to the following locations,
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/KernelGateway/{notebook-instance-name}</i>
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/JupyterServer/default</i>

If you want to find out the name of the underlying instance where this notebook is running, uncomment the following code cell and run it.

In [ ]:
'''
import json

notebook_name = ''
resource_metadata_path = '/opt/ml/metadata/resource-metadata.json'
with open(resource_metadata_path, 'r') as metadata:
    notebook_name = (json.load(metadata))['ResourceName']
print("Notebook instance name: '{}'".format(notebook_name))
'''

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  D. Organize imports <a id ='Organize%20imports'> </a>

Organize all the library and module imports for later use.

In [ ]:
import boto3
import sagemaker
import sys
from botocore.config import Config

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

Print the installed versions of some of the important libraries.

In [ ]:
logging.info("Python version : {}".format(sys.version))
logging.info("Boto3 version : {}".format(boto3.__version__))
logging.info("SageMaker Python SDK version : {}".format(sagemaker.__version__))

###  E. Set AWS Region and boto3 config <a id ='Set%20AWS%20Region%20and%20boto3%20config'> </a>

Get the current AWS Region (where this notebook is running) and the SageMaker Session. These will be used to initialize some of the clients to AWS services using the boto3 APIs.

<div class="alert alert-block alert-info">  
<b>Note:</b> All the AWS services used by this notebook except Amazon Bedrock Data Automation and Amazon S3 will use the current AWS Region. For those two services, follow the guidance in the next cell.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> At the time of writing this notebook, Amazon Bedrock Data Automation was only available in the <mark>us-west-2 (Oregon)</mark> AWS Region. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock Data Automation (BDA) client's region and/or endpoint URL parameters to that AWS Region. As BDA expects the Amazon S3 buckets to be in the same AWS Region, the S3 client's and/or endpoint URL parameters will be the same as the BDA client.
</div>



In [ ]:
# Get the AWS Region, SageMaker Session and IAM Role references
my_session = boto3.session.Session()
logging.info("SageMaker Session: {}".format(my_session))
my_iam_role = sagemaker.get_execution_role()
logging.info("Notebook IAM Role: {}".format(my_iam_role))
my_region = my_session.region_name
logging.info("Current AWS Region: {}".format(my_region))

# Explicity set the AWS Region for BDA and S3 clients
bda_region = "us-west-2"
s3_region = "us-west-2"

Set the timeout and retry configurations that will be applied to all the boto3 clients used in this notebook.

In [ ]:
# Increase the standard time out limits in the boto3 client from 1 minute to 3 minutes
# and set the retry limits
my_boto3_config = Config(
    connect_timeout = (60 * 3),
    read_timeout = (60 * 3),
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

###  F. Create common objects <a id='Create%20common%20objects'></a>

Specify the input and output S3 bucket names, and initialize other variables.

In [ ]:
# Specify the input Amazon S3 bucket name that will contain all the source files
# and the output S3 bucket name that will contain the files generated by BDA
s3_input_bucket_name = "<The name of the input S3 bucket>"
s3_output_bucket_name = "<The name of the output S3 bucket>"

# Specify the name of the DynamoDB table that will contain the metadata
ddb_table_name = "<The name of the DynamoDB table that will store the metadata>"

# Specify the names of the BDA components
image_blueprint_name = "FileTaggerImageBlueprint"
image_blueprint_desc = "This blueprint is to extract key information from general images to support image search."
image_doc_class = "general images"
image_blueprint_version_arn = ""
document_blueprint_name = "FileTaggerDocumentBlueprint"
document_blueprint_desc = "This document represents a letter sent from a person or company to another."
document_doc_class = "Letters"
document_blueprint_version_arn = ""
bda_project_name = "FileTaggerProject"

# Specify and create the required output directories
data_dir = os.path.join(os.getcwd(), "data")
os.makedirs(data_dir, exist_ok = True)

###  G. Check and create the metadata table <a id ='Check%20and%20create%20the%20metadata%20table'> </a>

Run the following cell to check and create the specified Amazon DynamoDB table in the current AWS Region. This table will contain the metadata for the objects stored in S3.

In [ ]:
# Create the Amazon DynamoDB clients
ddb_client = boto3.client("dynamodb", region_name = my_region, config = my_boto3_config)
ddb_resource = boto3.resource("dynamodb", region_name = my_region, config = my_boto3_config)

# Note: 'check_and_create_metadata_table' is available through ./scripts/helper_functions.py
check_and_create_metadata_table(ddb_client, ddb_table_name)

###  H. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
This notebook uses the IAM role attached to the underlying notebook instance.  To view the name of this role, run the following cell. This IAM role should have the following permissions,
1. Full access to Amazon Bedrock Data Automation (BDA).
2. Full access for BDA to read from the Amazon S3 bucket that contains the files to be tagged.
3. Full access for BDA to the Amazon S3 bucket that will contain the files generated by BDA.
4. Full access to this notebook to read and write to the input and output Amazon S3 buckets.
5. Full access to read and write to the Amazon DynamoDB table that will contain the metadata.
6. Access to write to Amazon CloudWatch Logs.

<div class="alert alert-block alert-info">
<b>Note:</b>  If you are running this notebook as part of a workshop session, by default, all these permissions will be setup.
</div>

Run the following cell to print the details of the IAM role attached to the underlying notebook instance.

In [ ]:
# Print the IAM role ARN and console URL
logging.info("This notebook's IAM role is '{}'".format(my_iam_role))
arn_parts = my_iam_role.split('/')
logging.info("Details of this IAM role are available at https://{}.console.aws.amazon.com/iamv2/home?region={}#/roles/details/{}?section=permissions"
             .format(my_region, my_region, arn_parts[len(arn_parts) - 1]))

### I. Setup BDA <a id ='Setup%20BDA'> </a>

Run the following cells to setup Amazon Bedrock Data Automation (BDA) components - blueprints, blueprint versions, and projects. The configuration for inferring content from document and image files will be defined in these components.

<div class="alert alert-block alert-info">
<b>Note:</b> At the time of writing this notebook, BDA supported only <b>PDF, JPEG, PNG, TIFF</b> file extensions for Document/Image Modality. So this notebook will ignore the processing of any other file present in the input S3 bucket.
</div>

In [ ]:
# Create the BDA clients
bda_client = boto3.client("bedrock-data-automation", region_name = bda_region, config = my_boto3_config)
bda_rt_client = boto3.client("bedrock-data-automation-runtime", region_name = bda_region, config = my_boto3_config)

In [ ]:
# Note: 'check_and_create_bda_blueprint', 'check_and_create_bda_blueprint_version' and 'check_and_create_bda_project'
# are available through ./scripts/helper_functions.py
image_blueprint_arn = check_and_create_bda_blueprint(bda_client, image_blueprint_name, 'IMAGE', image_blueprint_desc, image_doc_class)
image_blueprint_version_arn = check_and_create_bda_blueprint_version(bda_client, image_blueprint_arn, image_blueprint_version_arn)
document_blueprint_arn = check_and_create_bda_blueprint(bda_client, document_blueprint_name, 'DOCUMENT', document_blueprint_desc, document_doc_class)
document_blueprint_version_arn = check_and_create_bda_blueprint_version(bda_client, document_blueprint_arn, document_blueprint_version_arn)
bda_project_arn = check_and_create_bda_project(bda_client, bda_project_name,
                                               document_blueprint_arn, document_blueprint_version_arn,
                                               image_blueprint_arn, image_blueprint_version_arn)

## 2. Generate and store the file tags in the metadata table <a id ='Generate%20and%20store%20the%20file%20tags%20in%20the%20metadata%20table'> </a>

Running the following cell performs the following:

1. Gets the list of all the files in the input S3 bucket.
2. For each file, the file extension is checked to be one of these -> 'pdf', 'jpeg', 'png', 'tiff'. If not, that file will be ignored. If yes, then the following steps will be executed:
   1. Amazon Bedrock Data Automation (BDA) will be invoked which will,
       1. Read the specified file from the input S3 bucket.
       2. Processe the file as per the configuration in the BDA project.
       3. If there are any matching blueprints found in the project, they will also be applied; else the blueprints will be ignored.
       4. Store the output results (metadata file, standard output, custom output) in the output S3 bucket.
    2. From the BDA results, the required data fields will be extracted to create the file tags.
    3. The file tags will be stored as a JSON type metadata in the specified metadata (DynamoDB) table using the file's S3 URI as the key. 

In [ ]:
# Note: 'list_s3_files' is available through ./scripts/helper_functions.py
# Get the list of files that exist in the input S3 bucket
input_files = list_s3_files(s3_input_bucket_name, '')

In [ ]:
# Initialize the list to capture the processed files
processed_s3_uris = []

# Loop through the input files and process
for input_file in input_files:
    input_s3_file = '{}/{}'.format(s3_input_bucket_name, input_file)
    # Note: 'get_file_name_and_extension' is available through ./scripts/helper_functions.py
    input_file_name, input_file_extension = get_file_name_and_extension(input_file)
    # Filter out unsupported files
    if input_file_extension.lower() in ['pdf', 'jpeg', 'png', 'tiff']:
        # Invoke BDA on the input file
        logging.info("Processing file '{}'...".format(input_s3_file))
        # Note: 'process_bda_invocation' is available through ./scripts/helper_functions.py
        bda_invocation_id = process_bda_invocation(bda_rt_client,
                                                   input_s3_file,
                                                   s3_output_bucket_name,
                                                   bda_project_arn)
        if len(bda_invocation_id) == 0:
            logging.error("Could not complete processing file. Invocation id not available.")
        else:
            # Note: 'process_bda_invocation' is available through ./scripts/helper_functions.py
            file_metadata = process_bda_result(bda_invocation_id, s3_output_bucket_name, data_dir)
            # Insert/update the metadata table
            file_s3_uri = 's3://{}/{}'.format(s3_input_bucket_name, input_file)
            # Note: 'write_to_metadata_table' is available through ./scripts/helper_functions.py
            write_to_metadata_table(ddb_resource, ddb_table_name, file_s3_uri, file_metadata)
            # Add to the processed file list
            processed_s3_uris.append(file_s3_uri)
            logging.info("Completed processing file.")
    else:
        logging.warning("Ignored processing of unsupported file '{}'.".format(input_s3_file))

## 3. Retrieve the file tags from the metadata table <a id='Retrieve%20the%20file%20tags%20from%20the%20metadata%20table'></a>

Loop through all the processed files, retrieve their corresponding file tags from the metadata table and print them.

In [ ]:
# Loop through the processed files
for processed_s3_uri in processed_s3_uris:
    # Retrieve the metadata
    # Note: 'retrieve_from_metadata_table' is available through ./scripts/helper_functions.py
    logging.info("File: {}, Metadata = {}".format(processed_s3_uri, retrieve_from_metadata_table(ddb_resource,
                                                                                                 ddb_table_name,
                                                                                                 processed_s3_uri)))   

## 4. Cleanup <a id='Cleanup'></a>

As a best practice, you should delete AWS resources that are no longer required.  This will help you avoid incurring unncessary costs.

<div class="alert alert-block alert-info">
<b>Note:</b> If you are running this notebook as part of a workshop session, by default, all resources will be cleaned up at the end of the session. If you are running this notebook outside of a workshop session, you can cleanup the resources associated with this notebook by uncommenting the following code cell and running it.
</div>

Running the following cell will delete the following resources:
* The BDA blueprints and projects that were created.
* The DynamoDB table that was created to store the metadata.
* The files that were uploaded to the S3 bucket; not the S3 bucket itself.

In [ ]:
'''
# Note: 'delete_bda_blueprint' and 'delete_bda_project' available through ./scripts/helper_functions.py
delete_bda_project(bda_client, bda_project_arn)
delete_bda_blueprint(bda_client, image_blueprint_arn, image_blueprint_version_arn)
delete_bda_blueprint(bda_client, image_blueprint_arn, '')
delete_bda_blueprint(bda_client, document_blueprint_arn, document_blueprint_version_arn)
delete_bda_blueprint(bda_client, document_blueprint_arn, '')

# Note: 'delete_metadata_table' is available through ./scripts/helper_functions.py
delete_metadata_table(ddb_client, ddb_table_name)

# Create the Amazon S3 client
s3_client = boto3.client("s3", region_name = my_region, config = my_boto3_config)
# Loop through the downloaded files
for input_file in input_files:
    # Note: 'delete_from_s3' is available through ./scripts/helper_functions.py
    delete_from_s3(s3_client, s3_input_bucket_name, input_file)
'''

## 5. Conclusion <a id='Conclusion'></a>

We have now seen how to build a content-based file tagging system using Amazon Bedrock Data Automation (BDA) for document and image content. In the process, we also saw how easy it is to use BDA for custom data extraction from multi-modal content.

## 6. Frequently Asked Questions (FAQs) <a id='FAQs'></a>

**Q: What AWS services are used in this notebook?**

Amazon Bedrock, Amazon DynamoDB, Amazon S3, AWS Identity and Access Management (IAM), Amazon CloudWatch, and Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook depending on what you use to run the notebook.

**Q: Where can I access the FAQs for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/faqs/).

**Q: What models are supported by Amazon Bedrock?**

Go [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html).

**Q: Where can I find customer references for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/testimonials/).

**Q: Where can I find pricing information for the AWS services used in this notebook?**

- Amazon Bedrock pricing - go [here](https://aws.amazon.com/bedrock/pricing/).
- Amazon DynamoDB pricing - go [here](https://aws.amazon.com/dynamodb/pricing/).
- Amazon S3 pricing - go [here](https://aws.amazon.com/s3/pricing/).
- AWS Identity and Access Management (IAM) pricing - free.
- Amazon CloudWatch pricing - go [here](https://aws.amazon.com/cloudwatch/pricing/).
- Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook pricing - go [here](https://aws.amazon.com/sagemaker/pricing/).